# Introdução:

A partir do arquivo [Introdução](Introdução.ipynb), foi possível identificar parâmetros e características das Features e dos Targets. A partir disso, iremos treinar uma Rede Neural MLP (Multi Layer Perceptron), com o uso do Pytorch, com diversas arquiteturas distintas, através do Optuna, para identificar os melhores hiperparâmetros para a previsão dos nossos Targets.

As Redes Neurais funcionam a partir de Camadas de Neurônios, que recebem uma informação com um determinado peso e processa essa dado através de uma função de ativação e um viés, resultando num output que é passado para outro neurônio conectado ou final.
As Redes Neurais possuem numeros variáveis de camadas e neurônios em cada camada. Dessa forma, sendo hiperparâmetros da arquitetura da nossa rede.
 
Para isso, iremos usar o Optuna, que consegue calcular a métricas de diversas arquiteturas de Redes Neurais. Assim, escolhendo os hiperparâmetros que funcionam melhor para a predição dos nossos targets.

# Desenvolvimento:

## Importações e definições:

Importando as bibliotecas necessárias:

In [16]:
import pandas as pd
import numpy as np
import os
import optuna

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
import torch.optim as optim


Definindo as constantes do problema:

In [6]:
TAMANHO_TESTE = 0.1
TAMANHO_VALIDACAO = 0.1

SEMENTE_ALEATORIA = 1249

NUM_EPOCAS = 50

## Pré-processamento:

Abaixo serão descritas as etapas de pré-processamento explicadas no notebook anterior.

--- CORRIGIR, PRECISAMOS DIVIDIR O DATASET EM TREINO, VALIDAÇÃO E TESTE E DEPOIS NORMALIZAR AJUSTANDO AO TREINO E APLIANDO AO RESTO (VAMOS DESNORMALIZAR DEPOIS), MAS NÃO É BOM NORMALIZAR TODOS OS ATRIBUTOS

Carregando o dataset de energia:

In [7]:
TARGET = ["Storage_Efficiency_Percentage"]
FEATURES = [
    "Type_of_Renewable_Energy",
    "Installed_Capacity_MW",
    "Energy_Production_MWh",
    "Energy_Consumption_MWh",
    "Energy_Storage_Capacity_MWh",
    "Grid_Integration_Level",
    "Initial_Investment_USD",
    "Funding_Sources",
    "Financial_Incentives_USD",
    "GHG_Emission_Reduction_tCO2e",
    "Air_Pollution_Reduction_Index",
]

df_energia = pd.read_csv('energy_dataset_.csv')
df_energia = df_energia.reindex(FEATURES + TARGET, axis=1)
df_energia = df_energia.dropna()
df_energia

,Type_of_Renewable_Energy,Installed_Capacity_MW,Energy_Production_MWh,Energy_Consumption_MWh,Energy_Storage_Capacity_MWh,Grid_Integration_Level,Initial_Investment_USD,Funding_Sources,Financial_Incentives_USD,GHG_Emission_Reduction_tCO2e,Air_Pollution_Reduction_Index,Storage_Efficiency_Percentage
0,4,93.423205,103853.2206,248708.4892,2953.248771,4,4.732248e+08,1,9.207772e+06,6663.816572,81.742461,89.887562
1,4,590.468942,190223.0649,166104.1642,5305.174042,4,1.670697e+08,2,1.685101e+06,30656.049820,78.139042,84.403343
2,1,625.951142,266023.4824,424114.6308,2620.192622,2,8.463610e+07,2,5.111813e+06,1749.613759,8.461296,60.498249
3,1,779.998728,487039.5296,308337.7316,1925.250307,3,3.967690e+08,2,4.805902e+06,43233.237820,8.402441,86.897861
4,3,242.106837,482815.0856,360437.7705,3948.945383,2,3.574413e+07,1,1.668601e+07,14858.662760,28.822867,70.949351
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,3,745.032555,280007.5738,230544.8268,4351.687893,4,3.484136e+08,2,1.558508e+07,25234.911810,78.923200,90.791405
14996,1,15.187023,377340.5803,358547.3589,6792.194696,4,2.560179e+08,3,6.866618e+06,15762.519790,54.982974,78.252040
14997,3,877.539059,480497.3920,214441.6719,4588.725297,1,1.300112e+08,2,3.837764e+06,44597.809410,43.915897,58.282928
14998,7,551.264716,436383.1694,137043.8713,7251.144215,2,3.334831e+08,2,5.347706e+06,34363.858000,4.877145,73.573666


Aplicando a normalização por Máximo Absoluto considerando as colunas fixas, que contém dados discretos e necessários para o treinamento e interpretabilidade do algoritmo:

In [17]:
df_energia_normalizado = df_energia.copy()
colunas_fixas = ['Type_of_Renewable_Energy', 'Grid_Integration_Level','Funding_Sources']
colunas_variaveis = df_energia_normalizado.columns.difference(colunas_fixas)

normalizador = MaxAbsScaler()
df_energia_normalizado[colunas_variaveis] = normalizador.fit_transform(df_energia_normalizado[colunas_variaveis])
df_energia_normalizado

,Type_of_Renewable_Energy,Installed_Capacity_MW,Energy_Production_MWh,Energy_Consumption_MWh,Energy_Storage_Capacity_MWh,Grid_Integration_Level,Initial_Investment_USD,Funding_Sources,Financial_Incentives_USD,GHG_Emission_Reduction_tCO2e,Air_Pollution_Reduction_Index,Storage_Efficiency_Percentage
0,4,0.093425,0.207710,0.552781,0.295350,4,0.946562,1,0.460422,0.133283,0.817584,0.898921
1,4,0.590479,0.380453,0.369184,0.530563,4,0.334179,2,0.084261,0.613151,0.781543,0.844076
2,1,0.625962,0.532056,0.942639,0.262042,2,0.169292,2,0.255609,0.034994,0.084629,0.605013
3,1,0.780012,0.974096,0.685313,0.192541,3,0.793632,2,0.240313,0.864707,0.084041,0.869022
4,3,0.242111,0.965647,0.801110,0.394928,2,0.071497,1,0.834361,0.297188,0.288285,0.709529
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,3,0.745045,0.560025,0.512410,0.435206,4,0.696910,2,0.779310,0.504723,0.789386,0.907960
14996,1,0.015187,0.754694,0.796909,0.679278,4,0.512096,3,0.343356,0.315266,0.549937,0.782560
14997,3,0.877554,0.961012,0.476619,0.458912,1,0.260053,2,0.191902,0.891999,0.439245,0.582859
14998,7,0.551274,0.872782,0.304594,0.725176,2,0.667045,2,0.267405,0.687310,0.048781,0.735774


Dividindo os dados em treino, validação e teste, permitindo que a rede neural se ajuste aos dados de treino e seja otimizada para os dados de validação, evitando o *overfitting*:

In [9]:
# Separação dos dados de teste
indices = df_energia.index
indices_treino_val, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino_val = df_energia.loc[indices_treino_val]
df_teste = df_energia.loc[indices_teste]

X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values

In [10]:
# Separação dos dados de treino e validação
indices = df_treino_val.index
indices_treino, indices_val = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df_energia.loc[indices_treino]
df_val = df_energia.loc[indices_val]

X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values

X_val = df_val.reindex(FEATURES, axis=1).values
y_val = df_val.reindex(TARGET, axis=1).values

Fazendo uma normalização por Máximo Absoluto, ajustando aos dados de treino e transformando aos demais dados:

In [11]:
norm_x = MaxAbsScaler()
norm_x.fit(X_treino)

norm_y = MaxAbsScaler()
norm_y.fit(y_treino)

X_treino = norm_x.transform(X_treino)
y_treino = norm_y.transform(y_treino)

X_val = norm_x.transform(X_val)
y_val = norm_y.transform(y_val)

X_teste = norm_x.transform(X_teste)
y_teste = norm_y.transform(y_teste)


Convertendo os dados para Tensores, uma estrutura especial utilizada no módulo `Pytorch`:

In [12]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)

X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

## Optuna:

In [13]:
# Função que o Optuna vai otimizar
def objective(trial):
    # Hiperparâmetros a serem testados
    n_camadas = trial.suggest_int("n_layers", 1, 4)
    tamanho_camada_oculta = trial.suggest_int("hidden_size", 16, 128)
    funcao_ativacao = trial.suggest_categorical("activation", ["ReLU", "Tanh"])
    taxa_aprendizado = trial.suggest_float("lr", 1e-4, 1e-2)

    # Montando a rede com base nos hiperparâmetros
    layers = []
    input_size = X.shape[1]
    for i in range(n_camadas):
        layers.append(nn.Linear(input_size if i == 0 else tamanho_camada_oculta, tamanho_camada_oculta))
        layers.append(getattr(nn, funcao_ativacao)())
    layers.append(nn.Linear(tamanho_camada_oculta, 1))
    model = nn.Sequential(*layers)

    # Treinamento
    otimizador = optim.Adam(model.parameters(), lr=taxa_aprendizado)
    fun_perda = nn.MSELoss()
    
    for epoch in range(50):  
        model.train()
        y_pred = model(torch.tensor(X_treino, dtype=torch.float32))
        loss = fun_perda(y_pred, torch.tensor(y_teste, dtype=torch.float32))
        otimizador.zero_grad()
        loss.backward()
        otimizador.step()
    
    return loss.item()

In [14]:
study = optuna.create_study(direction="minimize")

[I 2025-06-03 11:23:00,265] A new study created in memory with name: no-name-4611a82f-a93f-4066-9080-e34be2e639af


In [15]:
# SO RODAR QUANDO FOR NECESSÁRIO

#study.optimize(objective, n_trials=100)  # Faz 100 testes de arquitetura

#print("Melhores hiperparâmetros:", study.best_params)

## Rede Neural MLP:

## Treinamento (Talvez):

## Curva de Aprendizado (Talvez):

## Testando a Rede:

# Conclusões:

# Referências:

[1] CASSAR, DANIEL. "". Material de aula, Ano

[2] 

[3] 